### Attention Is All You Need (2017) 실습

본 코드는 논문 Attention Is All You Need(2017)을 리뷰한 이후 transfomer의 아키텍처를 더욱 이해하고자 실제 실습한 내용을 담고 있다.   
리뷰 내용 :    
   





* ref : https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/Attention_is_All_You_Need_Tutorial_(German_English).ipynb

In [7]:
%pip install torchtext==0.6.0

     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ------- -------------------------------- 10.2/57.6 kB ? eta -:--:--
     --------------------------------- ---- 51.2/57.6 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 57.6/57.6 kB 430.1 kB/s eta 0:00:00
     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ---------------------------------------- 60.9/60.9 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/64.2 kB ? eta -:--:--
   ---------------------------------------- 64.2/64.2 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/16.2 MB ? eta -:--:--
   ---------------------------------------- 0.1/16.2 MB 3.6 MB/s eta 0:00:05
    --------------------------------------- 0.4/16.2 MB 3.8 MB/s eta 0:00:05
   - -------------------------------------- 0.5/16.2 MB 3.6 MB/s eta 0:00:05
   -- ----------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
%%capture
%pip install spacy
%python -m spacy download en
%python -m spacy download de

In [34]:
import spacy

python -m spacy download en_core_web_sm
python -m spacy download de_core_news_sm


SyntaxError: invalid syntax (3853461897.py, line 3)

In [ ]:
# 간단히 토큰화(tokenization) 기능 써보기
tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized):
    print(f"인덱스 {i}: {token.text}")